In [1]:
# switch to the project directory
%cd ..
# working directory should be ../FSE

c:\Users\admin\Desktop\research\FSE


In [2]:
from pdi.constants import (
    PARTICLES_DICT,
)

from pdi.data.types import Split

In [3]:
wandb_config = {
    "bs": 512,
    "max_epochs": 1,
    "start_lr": 5e-4,
    "embed_hidden": 32,
    "embed_out": 16,
    "ff_hidden": 32,
    "pool_hidden": 32,
    "num_heads": 1,
    "num_blocks": 5,
    "dropout": 0.1,
    "lr_patience": 2,
    "lr_factor": 0.5,
    "epoch_patience": 10,
}

wandb_config["embed_out"] -= wandb_config["embed_out"] % wandb_config["num_heads"]

In [4]:
import torch
import torch.nn as nn

torch.cuda.set_device(0)
device = torch.device("cuda")

In [5]:
from pdi.data.preparation import EnsemblePreparation
from pdi.models import NeuralNetEnsemble


data_preparation = EnsemblePreparation()
create_model = lambda: NeuralNetEnsemble(
    data_preparation.group_ids()[0],
    [wandb.config.embed_hidden, wandb.config.embed_out, 1],
    nn.ReLU,
    wandb.config.dropout,
).to(device)



In [6]:
import wandb
import os
from torch.profiler import profile, ProfilerActivity

experiment_name = "Profile_Final"
train_loader = data_preparation.prepare_dataloaders(wandb_config["bs"], 0, [Split.TRAIN])
target_code = 2212

save_path = f"reports/profile_final"
os.makedirs(save_path, exist_ok=True)


with wandb.init(project=experiment_name, config=wandb_config, name=PARTICLES_DICT[target_code], mode="offline") as run:
    pos_weight = torch.tensor(data_preparation.pos_weight(target_code)).float().to(device)
    wandb.log({"pos_weight": pos_weight.item()})
    model = create_model()
    optimizer = torch.optim.Adam(model.parameters(), lr=wandb.config.start_lr)
    loss_fun = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], with_stack=True, with_flops=True) as p:
        model.train()
        it = iter(train_loader)
        for i in range(10):
            input_data, targets, _, *group_id = next(it)
            input_data = input_data.to(device)
            binary_targets = (targets == target_code).type(torch.float).to(device)
            optimizer.zero_grad()
            out = model(input_data, *group_id).squeeze()
            loss = loss_fun(out, binary_targets)
            loss.backward()
            optimizer.step()

p.export_stacks(f"{save_path}/cpu.stacks", "self_cpu_time_total")
p.export_stacks(f"{save_path}/cuda.stacks", "self_cuda_time_total")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
C:\Users\admin\AppData\Roaming\Python\Python37\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


pos_weight,▁
pos_weight,48.35639


In [7]:
print(p.key_averages().table(row_limit=-1, max_name_column_width=120, sort_by="self_cpu_time_total"))

-------------------------------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                                           Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls   Total FLOPs  
-------------------------------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                                    aten::addmm        43.77%     375.616ms        43.77%     375.643ms     125.214ms     374.613ms        43.66%     374.622ms     124.874ms             3   1032192.000  
                                                       